# [Day 9] 실시간 손 추적과 상호작용

어제는 멈춰있는 사진에서 손을 찾았습니다.
오늘은 **실시간 웹캠 영상**에서 손을 찾고, 내 손가락 끝을 따라다니는 점을 만들어보겠습니다.
마치 마법사가 된 기분을 느끼실 수 있을 겁니다.

In [ ]:
import cv2 as cv
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 동영상 모드로 설정 (static_image_mode=False)
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

## 1. 랜드마크 콕 집어내기

손을 인식하면 `results.multi_hand_landmarks` 안에 21개의 점 정보가 들어옵니다.
그 중에서 **검지 손가락 끝(8번)**의 위치를 찾아봅시다.

In [ ]:
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break
    
    # 1. 좌우 반전 (거울모드) - 이걸 해야 움직이기 편합니다.
    frame = cv.flip(frame, 1)
    
    # 2. 색상 변환 및 손 인식
    image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    # 3. 손이 감지되었다면?
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 뼈대 그리기
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # --- 검지 손가락 끝(8번) 찾기 --- 
            # 랜드마크 좌표는 0~1 사이의 비율로 나옵니다. 화면 크기를 곱해줘야 진짜 좌표가 됩니다.
            h, w, c = frame.shape  # 화면의 높이(h), 너비(w)
            
            index_finger_tip = hand_landmarks.landmark[8]
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            
            # 검지 끝에 빨간 왕점 그리기
            cv.circle(frame, (cx, cy), 15, (0, 0, 255), -1)
            
            # 좌표 출력해보기
            cv.putText(frame, f"({cx}, {cy})", (cx+20, cy), 
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    cv.imshow('Hand Tracking', frame)
    if cv.waitKey(1) == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()

## 2. 코드 설명

- `cv.flip(frame, 1)`: 1은 좌우 반전을 의미합니다. 거울처럼 내 움직임과 똑같이 보여야 헷갈리지 않습니다.
- `landmark.x * w`: MediaPipe는 좌표를 "전체의 50% 지점(0.5)" 같이 알려줍니다. 그래서 전체 너비(`w`)를 곱해야 픽셀 좌표가 나옵니다.
- `int()`: 픽셀 좌표는 반드시 정수(1, 2, 3...)여야 합니다. 소수점을 버리기 위해 `int`로 감싸줍니다.

### [실습 1] 엄지 손가락에도 점 찍기

위 코드를 수정해서 **엄지 손가락 끝(4번)**에도 파란색 점을 찍어보세요.
두 손가락이 만나면(거리가 가까워지면) "Click!" 이라고 화면에 글씨를 띄울 수도 있을까요? (심화 도전 과제!)

In [ ]:
# 여기에 도전해보세요!


## 3. 마무리

이제 내 손의 위치를 정확하게 알아낼 수 있습니다.
이 좌표들을 로봇에게 전달하기만 하면, 로봇이 내 손을 따라 움직이게 되는 것이죠.

다음 시간에는 컴퓨터의 속도(FPS)를 측정하는 법을 배우고, 이 영상 처리 기술을 로봇 제어와 어떻게 연결할지 큰 그림을 그려보겠습니다.
수고하셨습니다!